In [1]:
import pandas as pd
import dill

In [2]:
with open('/Users/dolteanu/local_documents/Coding/MSc_github/Results/Ontario_gisaid_6/Ontario_gisaid_6_classification_report.pkl', 'rb') as data_path:
    classification_report = dill.load(data_path)

In [3]:
# initialize an empty dictionary to store the average metrics for each classifier
average_metrics = {}

# iterate over the classifiers
for classifier_name, reports in classification_report.items():
    # initialize an empty dictionary to store the average metrics for each class
    average_class_metrics = {}
    # iterate over the classification reports (list) for each fold
    for report in reports:
        for class_name, metrics in report.items():
            # if class_name!='accuracy':
                # metrics.pop('support')
                if class_name not in average_class_metrics:
                    average_class_metrics[class_name] = metrics
                elif class_name!='accuracy':
                    for metric, value in metrics.items():
                        average_class_metrics[class_name][metric] += value
                else:
                    average_class_metrics['accuracy'] += metrics
    # divide the accumulated values by the number of folds to obtain the average metrics
    num_folds = len(reports)
    for class_name, metrics in average_class_metrics.items():
        if class_name!='accuracy':
            for metric, value in metrics.items():
                average_class_metrics[class_name][metric] /= num_folds
        else:
            average_class_metrics['accuracy'] /= num_folds
    average_metrics[classifier_name] = average_class_metrics

In [9]:
table = pd.DataFrame.from_dict(average_metrics['LinearDiscriminant'])

In [10]:
table.drop(index='support')

,G,GH,GK,GR,GRY,GV,O,S,V,accuracy,macro avg,weighted avg
precision,0.995122,0.999815,0.999176,0.991525,0.970216,0.933333,0.975000,1.0,1.0,0.9934,0.984910,0.993539
recall,0.983159,0.999631,0.999794,0.980292,0.987586,0.950000,0.833333,1.0,1.0,0.9934,0.970422,0.993400
f1-score,0.989068,0.999723,0.999485,0.985859,0.978789,0.916667,0.875714,1.0,1.0,0.9934,0.971700,0.993393


In [6]:
export = pd.DataFrame()
for classifier, table in average_metrics.items():
    export.from_dict(table)
    

In [ ]:
with open('table.tex','w') as tf:
     tf.write(df.to_latex())